# Scraping Wikipedia Celebrity Deaths

Creating a notebook to produce the dataset found at the [Kaggle Celebrity Deaths Page](https://www.kaggle.com/hugodarwood/celebrity-deaths).

Attempting to replace the current dataset since it isn't complete (up-to-date) since there's no notebook to run to get up-to-date information and it has bad parses for some of the fields.

This notebook isn't perfect either but does a better job.

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import re
import requests
import json

from os import listdir
from os.path import isfile, join

from bs4 import BeautifulSoup as bs

In [3]:
month_to_num = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

Iterators:

In [4]:
year_list = range(2004,2017)
type(year_list)

list

### [DON'T RUN AGAIN] Scrape all monthly death pages

Scrape all death pages and store into the '../out/raw_pages' directory

In [228]:
"""
for year in year_list:
    for month in month_to_num.keys():
        filename = str(year) + '_' + str(month_to_num[month]) + '_deaths.json'
        url = ('https://en.wikipedia.org/w/api.php?action=query&titles=Deaths_in_' +  
               month + '_' + str(year) + 
               '&prop=revisions&rvprop=content&format=json')
        content = requests.get(url).json()
        with open("../out/raw_pages/" + filename, "wb") as outfile:
            json.dump(content, outfile)         
"""
''

''

### Helper functions

In [297]:
desc_death_re = re.compile('(.*?),? ?((?:.+)*?). (?:.*?)?', re.DOTALL|re.MULTILINE)
death_clean_no_url_re = re.compile('\s?(.+)[.]?(?:\<ref|\[?http)(?:.*)?$')

"""
Input: single text string to be processed

Output: list of strings of length two
  - first string is description of person
  - second string is cause of death 
  (last clause of input when more than one comma in field)
  
"""
def get_description_and_death(text):
    text_no_url = text
    if ('http' in text_no_url) or ('<ref' in text_no_url):
        try:
            text_no_url = death_clean_no_url_re.match(text).groups()[0]
        except AttributeError:
            print text
            raw_input("Press enter to continue")
            raw_input("Press enter to continue")
    text_parts = text_no_url.replace('=','').split(',')
    num_parts = len(text_parts)
    if num_parts == 0:
        return ['', '']
    elif num_parts == 1:
        return text_parts + ['']
    elif num_parts == 2:
        return [",".join(text_parts)] + [text_parts[1]]
    else:
        return ([",".join(text_parts[:-1])] + [text_parts[-1]])
                #[death_clean_no_url_re.match(text_parts[-1]).groups()[0]])

"""
Runs get_description_and_death on the last element of a list
"""
def add_description_and_death(entry_list):
    return entry_list[:-1] + get_description_and_death(entry_list[-1])

In [286]:
"""
Inputs: list, index number, character pattern to be replaced

Outputs: list with entry in index number cleaned of the character pattern to be replaced
"""
def clean_text(text_list, idx_num, chars):
    text_list_out = text_list
    text_list_out[idx_num] = text_list[idx_num].replace(chars, '')
    return text_list_out

In [287]:
mo_yr_key_re = re.compile('(\d+)_(\d+).*?')
name_age_re = re.compile('\[\[(.*?)\]\], (\d+), (.+)?$', re.MULTILINE)

"""
Inputs: month-year key string, text entry string

Outputs: list of month, year, name, and age
"""
def parse_month_year_name_age(my_key, text_entry):
    return (list(re.match(mo_yr_key_re, my_key).groups()) +
            list(re.match(name_age_re, text_entry.replace('\n', '')).groups()))

In [288]:
"""
Find links and convert them to the displayed text
"""
def remove_link_text(text_block):
    return

### Re-read and process scraped file data locally

In [289]:
jsonfiles = [f for f in listdir('../out/raw_pages/') if (isfile(join('../out/raw_pages/', f)) and f[-4:] == 'json')]

In [290]:
month_year_pages = {}

for jfilename in jsonfiles:
    my_key = "_".join(re.match(mo_yr_key_re, jfilename).groups())
    with open('../out/raw_pages/' + jfilename, 'rb') as infile:
        contents = infile.read()
        infile.close()
    month_year_pages[my_key] = json.loads(contents)['query']['pages'].values()[0]['revisions'][0]['*']

In [291]:
counter = 0
for value in month_year_pages.values():
    new_entries = len(value.encode('utf-8').rstrip().split('*'))
    counter += new_entries
counter

57927

Should combine cell below with cell above (later).

In [292]:
for my_key in month_year_pages.keys():
    month_year_pages[my_key] = [
        add_description_and_death(
            parse_month_year_name_age(my_key, entry)
        )
        for entry in month_year_pages[my_key].encode('utf-8').rstrip().split('*')
        if re.match(name_age_re, entry.replace('\n', ''))]

Make dataframe:

In [293]:
df_full = pd.DataFrame(columns=['year','month','name','age','desc','cause_of_death'])
print df_full.shape
df_full.head()

(0, 6)


,year,month,name,age,desc,cause_of_death


In [294]:
for entry in month_year_pages.values():
    df_sub = pd.DataFrame(entry, columns=['year','month','name','age','desc','cause_of_death'])
    df_full = pd.concat([df_full, df_sub], axis=0)

In [295]:
print df_full.shape
df_full.head()

(49051, 6)


,year,month,name,age,desc,cause_of_death
0,2014,10,Lynsey de Paul,64,"English singer-songwriter (""[[Won't Somebody D...",brain haemorrhage.<ref>[
1,2014,10,Maurice Hodgson|Sir Maurice Hodgson,94,British business executive.<ref>[,
2,2014,10,Shlomo Lahat,86,"Israeli general and politician, Mayor of [[Tel...",lung infection.<ref>[
3,2014,10,José Martínez (infielder)|José Martínez,72,Cuban baseball player ([[Pittsburgh Pirates]])...,[[Chicago Cubs]]) and executive ([[Atlanta Br...
4,2014,10,Oluremi Oyo,61,"Nigerian journalist, cancer.<ref>[",cancer.<ref>[


In [296]:
df_full.iloc[4].values

array(['2014', '10', 'Oluremi Oyo', '61',
       'Nigerian journalist, cancer.<ref>[', ' cancer.<ref>['], dtype=object)

### Further Processing

#### Extract Nationality

Extracting nationality text as well as possible by taking the first consecutive capitalized words in the description. 'Olympic' and similar capitalized words might throw this off.

In [191]:
natl_pattern1 = re.compile('((?:[A-Z][a-z\-]+ ?)+) ')

def get_nationality_text(desc_text):
    try:
        # get rid of url links
        return natl_pattern1.match(desc_text.strip('[')).groups()[0]
    except AttributeError, e:
        print desc_text

In [194]:
natl_pattern1.match('Native American asdf asdfasdf ').groups()

('Native American',)

## Further attributes outside of the death summary Wiki page

### Article length: Fame Score

Get Wikipedia URL for getting fame score i.e. size of article:

In [10]:
def get_wiki_url(name_text):
    return name_text.split('|')[0].strip('[').strip(']')

In [11]:
df_entries['name'] = df_entries.name.map(get_wiki_url)

Get page metadata:

In [12]:
base_url_prefix = 'https://en.wikipedia.org/w/api.php?action=query&format=json&titles=' 
base_url_suffix = '&prop=revisions&rvprop=size'

In [13]:
def get_page_size(wiki_name):
    size_url = base_url_prefix + wiki_name.replace(' ','_') + base_url_suffix
    size_page = requests.get(size_url)
    return size_page.json()['query']['pages'].values()[0]['revisions'][0]['size']

Clean up wiki link text

In [15]:
def remove_wiki_url_delims(text):
    return text.replace('[[','').replace(']]','')

### Birth and death data

Getting birthday data. It's probably helpful to see [Stack Overflow thread](https://stackoverflow.com/questions/12250580/parse-birth-and-death-dates-from-wikipedia)

In [20]:
text_for_search = '         = Adele Faccio\n| honorific-suffix    =\n| image               =\n| caption             =\n| constituency_MP     =<!-- Can be repeated up to eight times by adding a number -->\n| parliament          =<!-- Can be repeated up to eight times by adding a number -->\n| majority            =<!-- Can be repeated up to eight times by adding a number -->\n| term_start          =<!-- Can be repeated up to eight times by adding a number -->\n| term_end            =<!-- Can be repeated up to eight times by adding a number -->\n| predecessor         =<!-- Can be repeated up to eight times by adding a number -->\n| successor           =<!-- Can be repeated up to eight times by adding a number -->\n| birth_date          ={{birth date|1920|11|13|df=y}} \n| birth_place         =Pontebba, Udine\n| death_date          ={{death date and age|2007|02|08|1920|11|13}}\n| death_place         =Rome\n| nationality         ={{ITA}}\n| party               = Radical Party (Partito Radicale)\n| otherparty          = <!--For additional political affiliations -->\n| spouse              =\n| partner             = <!--For those with a domestic partner and not married -->\n| relations           =\n| children            =\n| residence           =\n| alma_mater          =\n| occupation          =\n| profession          =\n| religion            =\n| signature           =\n| website             =\n| footnotes           =\n}}\n\n\'\'\'Adele Faccio\'\'\' (November 13, 1920 in [[Pontebba]], [[Udine]] \xe2\x80\x93 February 8, 2007 in [[Rome]]) was an [[Italy|Italian]] [[politician]] and deputy of the [[Radical Party (Italy)|Radical Party]] (\'\'Partito Radicale\'\').<ref>{{It icon}} "[http://www.corriere.it/Primo_Piano/Cronache/2007/02_Febbraio/09/adelefaccio.shtml Morta la radicale Adele Faccio]." (February 9, 2007). \'\'Corriere della Sera.\'\' Retrieved June 21, 2007.</ref>'

In [8]:
base_bday_prefix = 'https://en.wikipedia.org/w/api.php?action=query&format=json&titles='
base_bday_suffix = '&prop=revisions&rvprop=content&rvsection=0'

def check_if_number(elem):
    try:
        return type(int(elem)) == int
    except ValueError:
        return False

    
def get_only_ints(date_list):
    return [e for e in date_list if check_if_number(e) > 0]


no_birth_death_urls = []

def get_life_death(name, extract_bday=True):
    try:        
        entry_url = base_bday_prefix + name.replace(' ','_') + base_bday_suffix
        entry_text = requests.get(entry_url).json()['query']['pages']
        entry_text = str(entry_text.values()[0]['revisions'][0]['*'].encode('utf-8'))
        
        if extract_bday:
            pattern = re.compile('.*?\{{2}(?:B|b)irth (?:D|d)ate(.+?)\}{2}', 
                                 re.MULTILINE|re.DOTALL)
        else:
            pattern = re.compile('.*?\{{2}(?:D|d)eath (?:D|d)ate(.+?)\}{2}', 
                                 re.MULTILINE|re.DOTALL)

        date_re = re.match(pattern, entry_text)
        date_data = "-".join(get_only_ints(date_re.groups()[0].strip(" ").split('|'))[:3])
        
        #print date_data
        return date_data
    
    except AttributeError, e:
        no_birth_death_urls.append(entry_url)

Birthday

Deathday

## Write out file

In [28]:
df_full.to_csv('../out/celeb_deaths_wikipedia_' +
                  month_select + '_' + str(year_select) + '.csv', 
                  index=False)

It would be useful to plot missing data by page_size i.e. fame or "importance."